## Bikes

In [82]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [75]:
veligo = pd.read_csv('./data/bikes/souscripteurs-veligo-collectivites.csv', sep=';')

In [76]:
# Determine what to keep
keep_deps = ['75', '92', '93', '94'] # departments to keep, first is paris, latter three are first ring
all_geos = set(list(veligo.code_postal.unique()))
keep = set(geo for geo in all_geos if str(geo)[:2] in keep_deps) # first two characters in geocode correspond to department
i_keep = veligo.loc[veligo.code_postal.isin(keep)].index

# Drop the rest
i_drop = veligo.index.difference(i_keep)
before = len(veligo)
veligo.drop(index=i_drop, inplace=True)
print(f'Dropped {before-len(veligo)} of {before} rows, {len(veligo)} remaining')

Dropped 725 of 872 rows, 147 remaining


In [77]:
# Translate department numbers to label
label_dict = {'75': 'City',
            '92': 'Ring',
            '93': 'Ring',
            '94': 'Ring'}

# Apply labels, 'External' if one side of the commute is not in either City or Ring
for i, geo in veligo.code_postal.items():
    # Label City and Ring
    label_col = 'zone'
    dep = str(geo)[:2]
    if dep in keep_deps:
        veligo.at[i, label_col] = label_dict[dep]

In [86]:
warnings.simplefilter(action='ignore', category=FutureWarning)
veligo.nombre_de_souscriptions_jo.replace('<5', '0',inplace=True)
veligo.nombre_de_souscriptions_cargo.replace('<5', '0',inplace=True)
veligo['total_subs'] = veligo[['nombre_de_souscriptions_vae_classique', 'nombre_de_souscriptions_cargo',
                               'nombre_de_souscriptions_jo']].astype(int).sum(axis=1)
veligo.groupby(by='zone')['total_subs'].sum()

zone
City    48972
Ring    43344
Name: total_subs, dtype: int64